In [2]:
using DataFrames, CSV, StringEncodings, Dictionaries, Dates, StatsBase, HTTP, JSON, ProgressBars

In [12]:
yr = 2019
datadir = joinpath("..", "Data", string(yr))

"../Data/2019"

In [13]:
datadf = DataFrame(CSV.File(joinpath(datadir, "pitching_data.csv")))
resdf = DataFrame(CSV.File(joinpath(datadir, "game_results.csv")))

totaldf = leftjoin(datadf, resdf, on=:game_pk)
CSV.write(joinpath(datadir, "pitching_data_with_results.csv"), totaldf)

"../Data/2019/pitching_data_with_results.csv"

In [14]:
# Here we prepare the raw SP data frames to merge them correctly

spnames = DataFrame(CSV.File(joinpath(datadir, "SP_names.csv")))
spteams = DataFrame(CSV.File(joinpath(datadir, "SP_teams.csv")))
datadf = DataFrame(CSV.File(joinpath(datadir, "pitching_data_with_results.csv")))

spnames.join_key = string.(spnames.game_pk) .* string.(spnames.pitches) .* string.(spnames.babip) .* string.(spnames.launch_speed) .* string.(spnames.launch_angle)
spteams.join_key = string.(spteams.game_pk) .* string.(spteams.pitches) .* string.(spteams.babip) .* string.(spteams.launch_speed) .* string.(spteams.launch_angle)

a = size(spnames, 1)
b = size(spteams, 1)

println("Height of each DF: names = $a, teams = $b")

spnames = spnames[!, [:join_key, :game_pk, :player_id, :player_name]]
spteams = spteams[!, [:join_key, :player_id, :player_name]]
rename!(spteams, [:player_id, :player_name] .=> [:team_id, :team_name])

spdf = innerjoin(spnames, spteams, on=:join_key)
c = size(spdf, 1)

println("Height of joined df: $c")

if a == b && b == c
    spdf.join_key = string.(spdf.game_pk) .* string.(spdf.team_id)
    rename!(spdf, [:player_id, :player_name] .=> [:spitcher_id, :spitcher_name])
    spdf = spdf[!, [:join_key, :spitcher_id, :spitcher_name]]
    
    datadf.join_key = string.(datadf.game_pk) .* string.(datadf.player_id)
    leftjoin!(datadf, spdf, on=:join_key)
end

Height of each DF: names = 4858, teams = 4858
Height of joined df: 4858


Row,pitches,player_id,player_name,game_date,game_pk,total_pitches,pitch_percent,ba,iso,babip,slg,woba,xwoba,xba,hits,abs,launch_speed,launch_angle,spin_rate,velocity,effective_speed,whiffs,swings,takes,eff_min_vel,release_extension,pos3_int_start_distance,pos4_int_start_distance,pos5_int_start_distance,pos6_int_start_distance,pos7_int_start_distance,pos8_int_start_distance,pos9_int_start_distance,pitcher_run_exp,run_exp,bat_speed,swing_length,pa,bip,singles,doubles,triples,hrs,so,k_percent,bb,bb_percent,api_break_z_with_gravity,api_break_z_induced,api_break_x_arm,api_break_x_batter_in,hyper_speed,bbdist,hardhit_percent,barrels_per_bbe_percent,barrels_per_pa_percent,release_pos_z,release_pos_x,plate_x,plate_z,obp,barrels_total,batter_run_value_per_100,xobp,xslg,pitcher_run_value_per_100,xbadiff,xobpdiff,xslgdiff,wobadiff,swing_miss_percent,arm_angle,attack_angle,attack_direction,swing_path_tilt,rate_ideal_attack_angle,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches,GameDate,HomeID,HomeRuns,AwayID,AwayRuns,join_key,spitcher_id,spitcher_name
,Int64,Int64,String3,Date,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64?,Int64,Int64,Float64?,Float64?,Int64?,Float64?,Float64?,Int64,Int64,Int64,Float64?,Float64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Float64,Float64,Missing,Missing,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Float64,Float64?,Float64?,Float64?,Float64?,Float64?,Int64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64,Int64?,Float64,Float64?,Float64?,Float64,Float64?,Float64?,Float64?,Float64,Float64,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Date,Int64,Int64,Int64,Int64,String,Int64?,String31?
1,321,109,AZ,2019-09-24,564909,321,100.0,0.188,0.101,0.262,0.29,0.227,0.216,0.179,13,69,88.2,15.0,2203,87.7,87.17,47,165,156,-0.5,5.94,108,154,116,150,305,320,303,6.897,-6.897,missing,missing,73,44,10,1,0,2,25,34.2,4,5.5,2.59159,0.46072,0.21638,-0.07336,90.6,150,32.4324,5.40541,2.73973,5.69,-1.78,-0.01,1.99,0.233,2,-2.1486,0.225,0.264,2.1486,0.009,0.008,0.026,0.011,28.5,missing,missing,missing,missing,missing,missing,missing,2019-09-24,109,3,138,2,564909109,502190,"Leake, Mike"
2,294,137,SF,2019-09-17,565102,295,99.7,0.246,0.123,0.31,0.368,0.315,0.336,0.27,14,57,89.7,6.7,2302,88.8,88.49,27,117,177,-0.3,6.08,112,147,117,149,271,325,308,1.527,-1.527,missing,missing,68,44,10,2,1,1,14,20.6,10,14.7,2.37299,0.58925,0.22391,-0.19276,92.0,173,38.4615,2.5641,1.47059,5.81,-1.16,0.1,2.22,0.358,1,-0.519388,0.379,0.388,0.519388,-0.024,-0.021,-0.02,-0.021,23.1,missing,missing,missing,missing,missing,missing,missing,2019-09-17,111,6,137,7,565102137,657277,"Webb, Logan"
3,291,108,LAA,2019-07-25,564784,291,100.0,0.277,0.123,0.34,0.4,0.304,0.292,0.235,18,65,87.8,16.3,2270,87.5,86.78,50,149,142,-0.7,5.75,114,148,112,147,297,314,284,-1.78,1.78,missing,missing,70,49,14,2,0,2,17,24.3,4,5.7,2.42739,0.65876,0.30069,-0.12062,91.9,177,37.2093,6.97674,4.28571,5.89,-0.16,0.05,2.13,0.314,3,0.611684,0.281,0.39,-0.611684,0.042,0.033,0.01,0.012,33.6,missing,missing,missing,missing,missing,missing,missing,2019-07-25,108,8,110,10,564784108,660761,"Suarez, José"
4,283,137,SF,2019-09-24,566546,283,100.0,0.207,0.276,0.212,0.483,0.349,0.338,0.2,12,58,88.8,12.9,2327,88.0,87.58,49,134,149,-0.4,6.03,108,150,113,147,294,325,296,-1.249,1.249,missing,missing,71,39,6,1,0,5,20,28.2,11,15.5,2.35979,0.66138,0.16569,0.05722,92.3,181,42.8571,14.2857,7.04225,5.89,0.78,-0.15,2.13,0.343,5,0.441343,0.337,0.446,-0.441343,0.007,0.006,0.037,0.011,36.6,missing,missing,missing,missing,missing,missing,missing,2019-09-24,137,5,115,8,566546137,518516,"Bumgarner, Madison"
5,280,110,BAL,2019-07-25,564784,280,100.0,0.196,0.107,0.222,0.304,0.29,0.34,0.246,11,56,89.1,14.2,2148,86.8,86.11,24,106,174,-0.7,5.75,106,157,116,146,309,321,304,2.462,-2.462,missing,missing,69,46,7,3,0,1,11,15.9,11,15.9,2.64871,0.61144,0.54668,-0.1583,91.9,146,34.8837,4.65116,2.89855,5.47,-1.02,0.06,2.14,0.33

In [15]:
# Add on winning team

HomeTeamWin = datadf.HomeRuns .> datadf.AwayRuns
IsHomeTeam = datadf.player_id .== datadf.HomeID

datadf.winner = .!(xor.(HomeTeamWin, IsHomeTeam))

sum(ismissing.(datadf.spitcher_id))

0

In [16]:
datadf = coalesce.(datadf, NaN)

CSV.write(joinpath(datadir, "PitchingDataWithResultsAndStarters.csv"), datadf)

"../Data/2019/PitchingDataWithResultsAndStarters.csv"

In [17]:
homedf = datadf[datadf.player_id .== datadf.HomeID, :]
sort!(homedf, [:game_pk])
awaydf = datadf[datadf.player_id .== datadf.AwayID, :]
sort!(awaydf, [:game_pk])

sort!(datadf, [:game_pk])

datadf.offensive_xwoba = NaN .* zeros(size(datadf, 1))
datadf.offensive_launch_angle = NaN .* zeros(size(datadf, 1))
datadf.offensive_launch_speed = NaN .* zeros(size(datadf, 1))
datadf.offensive_hardhit_percent = NaN .* zeros(size(datadf, 1))

datadf[datadf.player_id .== datadf.HomeID, [:offensive_xwoba, 
                        :offensive_launch_angle, :offensive_launch_speed, 
                                                :offensive_hardhit_percent]] = Array(awaydf[!, [:xwoba, :launch_angle,
                                                                                    :launch_speed, :hardhit_percent]]);

datadf[datadf.player_id .== datadf.AwayID, [:offensive_xwoba, 
                        :offensive_launch_angle, :offensive_launch_speed, 
                                                :offensive_hardhit_percent]] = Array(homedf[!, [:xwoba, :launch_angle,
                                                                                    :launch_speed, :hardhit_percent]]);


In [18]:
# Here let's add on a last five game offense / last game with pitcher 

last5_o_xwoba = NaN .* ones(size(datadf, 1))
last5_o_launch_angle = NaN .* ones(size(datadf, 1))
last5_o_launch_speed = NaN .* ones(size(datadf, 1))
last5_o_hardhit_percent = NaN .* ones(size(datadf, 1))

last_p_xwoba = NaN .* ones(size(datadf, 1))
last_p_launch_angle = NaN .* ones(size(datadf, 1))
last_p_launch_speed = NaN .* ones(size(datadf, 1))
last_p_hardhit_percent = NaN .* ones(size(datadf, 1))

idx = 1

for r = ProgressBar(eachrow(datadf))
    last5df = datadf[(datadf.game_date .<= r.game_date) .& 
                    (datadf.player_id .== r.player_id) .&
                    (datadf.game_pk .!= r.game_pk), :]
    sort!(last5df, :game_date)
    
    last5df = last(last5df, 5)

    last5_o_hardhit_percent[idx] = mean(last5df.offensive_hardhit_percent)
    last5_o_launch_angle[idx] = mean(last5df.offensive_launch_angle)
    last5_o_launch_speed[idx] = mean(last5df.offensive_launch_speed)
    last5_o_xwoba[idx] = mean(last5df.offensive_xwoba)

    lastpitchdf = datadf[(datadf.game_date .<= r.game_date) .& 
                    (datadf.player_id .== r.player_id) .&
                    (datadf.game_pk .!= r.game_pk) .& 
                    (datadf.spitcher_id .== r.spitcher_id), :]

    sort!(last5df, :game_date)
    last5df = last(last5df, 1)     
    
    last_p_hardhit_percent[idx] = mean(lastpitchdf.hardhit_percent)
    last_p_launch_angle[idx] = mean(lastpitchdf.launch_angle)
    last_p_launch_speed[idx] = mean(lastpitchdf.launch_speed)
    last_p_xwoba[idx] = mean(lastpitchdf.xwoba)

    idx = idx + 1
end

datadf.last5_o_hardhit_percent = last5_o_hardhit_percent
datadf.last5_o_launch_angle = last5_o_launch_angle
datadf.last5_o_launch_speed = last5_o_launch_speed
datadf.last5_o_xwoba = last5_o_xwoba

datadf.last_p_hardhit_percent = last_p_hardhit_percent
datadf.last_p_launch_angle = last_p_launch_angle
datadf.last_p_launch_speed = last_p_launch_speed
datadf.last_p_xwoba = last_p_xwoba



0.0%┣                                            ┫ 0/4.9k [00:00<00:-28, -0s/it]
0.0%┣                                         ┫ 1/4.9k [00:00<Inf:Inf, InfGs/it]
9.7%┣████                                     ┫ 470/4.9k [00:00<00:04, 1.2kit/s]
20.9%┣████████▏                              ┫ 1.0k/4.9k [00:00<00:02, 2.4kit/s]
31.9%┣████████████▌                          ┫ 1.6k/4.9k [00:00<00:01, 3.2kit/s]
43.2%┣████████████████▉                      ┫ 2.1k/4.9k [00:01<00:01, 3.9kit/s]
54.4%┣█████████████████████▏                 ┫ 2.6k/4.9k [00:01<00:00, 4.5kit/s]
65.7%┣█████████████████████████▋             ┫ 3.2k/4.9k [00:01<00:00, 5.1kit/s]
77.0%┣██████████████████████████████         ┫ 3.7k/4.9k [00:01<00:00, 5.5kit/s]
88.8%┣██████████████████████████████████▋    ┫ 4.3k/4.9k [00:01<00:00, 5.9kit/s]
100.0%┣██████████████████████████████████████┫ 4.9k/4.9k [00:01<00:00, 6.2kit/s]
100.0%┣██████████████████████████████████████┫ 4.9k/4.9k [00:01<00:00, 6.2kit/s]


4866-element Vector{Float64}:
 0.266
 0.385
 0.358
 0.33
 0.322
 0.275
 0.342
 0.37
 0.2885
 0.298
 ⋮
 0.3238666666666667
 0.26533333333333337
 0.29128571428571426
 0.33794117647058824
 0.3325199999999999
 0.3775
 0.3216896551724138
 0.26328571428571435
 0.31816666666666665

In [19]:
CSV.write(joinpath(datadir, "dataset_with_past_performance.csv"), datadf)

"../Data/2019/dataset_with_past_performance.csv"

This part joins on the odds data now

In [20]:
dataperf = DataFrame(CSV.File(joinpath(datadir, "dataset_with_past_performance.csv")))
oddsdf = DataFrame(CSV.File(joinpath(datadir, "odds_data.csv")))

oddsdf.date = Date.(oddsdf.date, "mm/dd/yy") + Dates.Year(2000)
# dataperf.game_date = Date.(dataperf.game_date, "mm/dd/yy") + Dates.Year(2000)

teammap = DataFrame(CSV.File(joinpath(datadir, "..", "TeamNameMap.csv")))
applymap = Dictionary(teammap.concise, teammap.code)

oddsdf.HomeID = getindices(applymap, oddsdf.home)
oddsdf.AwayID = getindices(applymap, oddsdf.away)

dataperf.join_key = Dates.format.(dataperf.game_date, "yyyymmdd") .* string.(Int.(dataperf.HomeID)) .* 
                        string.(Int.(dataperf.AwayID)) .* string.(Int.(dataperf.HomeRuns)) .* string.(Int.(dataperf.AwayRuns))

oddsdf.join_key = Dates.format.(oddsdf.date, "yyyymmdd") .* string.(Int.(oddsdf.HomeID)) .*
                    string.(Int.(oddsdf.AwayID)) .* string.(Int.(oddsdf.home_score)) .* string.(Int.(oddsdf.away_score))

a = size(dataperf, 1)
b = size(oddsdf, 1)

println("Rows in dataperf: $a, Rows in oddsdf: $b")

total = leftjoin(dataperf, oddsdf[!, [:join_key, :home_opener_odds, :away_opener_odds]], on=:join_key)

c = size(total, 1)

println("Rows after join: $c")


CSV.write(joinpath(datadir, "dataset_with_odds_and_performance.csv"), total)


counts = countmap(total.join_key)

for (v, c) in counts
    if c >= 3
        println(v)
    end 
end

Rows in dataperf: 4866, Rows in oddsdf: 2935
Rows after join: 4866
2019051911613337
2019080711111854
2019052714511821
2019061312113845
